The Following shows code to scrape the information about best selling history book on Amazon. 
Then those results got saved into a csv to perform data processing.

In [75]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup
my_url = 'https://www.amazon.co.uk/Best-Sellers-Books-History/zgbs/books/65/ref=zg_bs_nav_b_1_b'
uClient = uReq(my_url)
html_doc =uClient.read()
uClient.close()

soup = BeautifulSoup(html_doc, 'html.parser')

# Grab all books in top 50 History book ranking in Amazon UK
container = soup.findAll("ol", {"class":"a-ordered-list a-vertical"})
ranking = 0
for book in container[0]:
    print("-----------------------------------")
    ranking += 1
# Find the Title of the Book  
    title = book.a.div.img['alt']
# Find the Author of the Book  
    author = book.find('a', {"class":"a-size-small a-link-child"})
#   If not, means there has not yet been a link for the author.
    if not author:
        author = book.find('span', {"class": "a-size-small a-color-base"}).text
    else:
        author = author.text
    price = book.find('span', {"class": "a-size-base a-color-price"}).text

# Find the Rating of the Book
    rating = book.div.i
    if rating:
        rating = rating.span.text
        numberRated = book.find('a', {"class": "a-size-small a-link-normal"}).text

#     Print them out
    print("Ranking: " + str(ranking))
    print("Title: " + title)
    print("Author: " + author)
    print("Price: " + price)
    if rating:
        print("Rating: {} ({})".format(rating, numberRated))
    else:
        print("Rating: None")

-----------------------------------
Ranking: 1
Title: 52 Times Britain was a Bellend: The History You Didn’t Get Taught At School
Author: James Felton
Price: £0.99
Rating: 4.5 out of 5 stars (526)
-----------------------------------
Ranking: 2
Title: Finding Freedom: Harry and Meghan and the Making of a Modern Royal Family
Author: Omid Scobie
Price: £13.60
Rating: None
-----------------------------------
Ranking: 3
Title: The Romanovs: 1613-1918
Author: Simon Sebag Montefiore
Price: £6.99
Rating: 4.5 out of 5 stars (787)
-----------------------------------
Ranking: 4
Title: The History of Ancient Egypt
Author: Bob Brier
Price: £0.00 Free with Audible trial
Rating: 4.8 out of 5 stars (5)
-----------------------------------
Ranking: 5
Title: Why I’m No Longer Talking to White People About Race: The #1 Sunday Times Bestseller
Author: Reni Eddo-Lodge
Price: £6.29
Rating: 4.6 out of 5 stars (1,955)
-----------------------------------
Ranking: 6
Title: Black and British: A Forgotten History


In [76]:
# Write down into CSV format and possibly clean the data
f = open("historyBooksData.csv", "w")
headers = "Ranking,Title,Author,Price,Rating\n"
f.write(headers)
ranking = 0
for book in container[0]:
    ranking += 1
    title = book.a.div.img['alt']
# Find the Author of the Book  
    author = book.find('a', {"class":"a-size-small a-link-child"})
#   If not, means there has not yet been a link for the author.
    if not author:
        author = book.find('span', {"class": "a-size-small a-color-base"}).text
    else:
        author = author.text
    price = book.find('span', {"class": "a-size-base a-color-price"}).text

# Find the Rating of the Book
    rating = book.div.i
    if rating:
        rating = rating.span.text
        numberRated = book.find('a', {"class": "a-size-small a-link-normal"}).text
    else:
        rating = "N/A"
    f.write(str(ranking) + "," + title.replace(",", "|") + "," + author + "," + price + "," + rating + "\n")
f.close()

In [79]:
# Manipulate and clean the data
import numpy as np
import pandas as pd

# Read the CSV
df = pd.read_csv("historyBooksData.csv", index_col = 0)
# Preview How the dataframe looks like before processing
print(df.head(3))
# Get rid of rows that has no rating (Drop those rows with NaN values)
df_clean = df
df_clean.dropna(subset = ['Rating'], inplace = True)

# Set the rating in a better format (From x.x out of 5 stars => x.x/5)
# If use a lambda function
df_clean['Rating'] = df_clean['Rating'].map(lambda x: x[:3] + '/5')

# Write and save the new clean data
df_clean.to_csv("cleanHistoryBookData.csv")

                                                     Title  \
Ranking                                                      
1        52 Times Britain was a Bellend: The History Yo...   
2        Finding Freedom: Harry and Meghan and the Maki...   
3                                  The Romanovs: 1613-1918   

                         Author   Price              Rating  
Ranking                                                      
1                  James Felton   £0.99  4.5 out of 5 stars  
2                   Omid Scobie  £13.60                 NaN  
3        Simon Sebag Montefiore   £6.99  4.5 out of 5 stars  


In [80]:
# Read the new dataframe and print it
clean_df = pd.read_csv("cleanHistoryBookData.csv", index_col = 0)
print(clean_df)

                                                     Title  \
Ranking                                                      
1        52 Times Britain was a Bellend: The History Yo...   
3                                  The Romanovs: 1613-1918   
4                             The History of Ancient Egypt   
5        Why I’m No Longer Talking to White People Abou...   
6                   Black and British: A Forgotten History   
8        The Five: The Untold Lives of the Women Killed...   
9        Too Much and Never Enough: How My Family Creat...   
10       Too Much and Never Enough: How My Family Creat...   
11       Natives: Race and Class in the Ruins of Empire...   
12       Why I'm No Longer Talking to White People Abou...   
13                           The Ritz London: The Cookbook   
14                        Alexander Hamilton (Great Lives)   
15                   Sapiens: A Brief History of Humankind   
17       The Five: The Untold Lives of the Women Killed...   
18      